In [5]:
import json

In [ ]:
json_data = []

with open("search_index.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f, start=1):
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
            json_data.append(obj)
        except json.JSONDecodeError as e:
            print(f"❌ Error parsing line {i}: {e}")

print("Total records:", len(json_data))
print("First record:", json_data[0])


In [ ]:
json_data[10]

{'url': 'https://github.com/github',
 'title': 'GitHub · GitHub',
 'description': 'How people build software. GitHub has 531 repositories available. Follow their code on GitHub.',
 'keywords': ['github', 'source', 'security'],
 'visible_text': "Navigation Menu Search code, repositories, users, issues, pull requests... Provide feedback We read every piece of feedback, and take your input very seriously. Saved searches Use saved searches to filter your results more quickly To see all available qualifiers, see our . GitHub We've verified that the organization controls the domain: Hey, this is us 👋 Yes, we are building GitHub on GitHub. In fact, we’ve been doing this since . That's when we made our first commit. Since then we pushed , opened , submitted roughly across from over . 🤯 But that's just us. We are proud to be part of the work of millions of developers, companies and robots across the solar system. 🪐 Yes, ! 🍿 An interconnected community The open source community is the 💗 heart of

In [ ]:
title = json_data[0]['title']
description = json_data[0]['description']
keywords = json_data[0]['keywords']
visible_text = json_data[0]['visible_text']
full_text = f'{title} {description} {keywords} {visible_text}'
full_text

"GitHub Docs Get started, troubleshoot, and make the most of GitHub. Documentation for new users, developers, administrators, and all of GitHub's products. ['github', 'secure', 'started', 'copilot', 'project'] GitHub Docs Help for wherever you are on your GitHub journey. Get started Collaborative coding GitHub Copilot CI/CD and DevOps Security and quality Client apps Project management Enterprise and teams Developers Community More docs Getting started At the heart of GitHub is an open-source version control system (VCS) called Git. Git is responsible for everything GitHub-related that happens locally on your computer. You can connect to GitHub using the Secure Shell Protocol (SSH), which provides a secure channel over an unsecured network. You can create a repository on GitHub to store and collaborate on your project's files, then manage the repository's name and location. Create sophisticated formatting for your prose and code on GitHub with simple syntax. Popular Pull requests let y

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # lowercase
    text = text.lower()

    # remove punctuation & numbers
    text = re.sub(r"[^a-z\s]", " ", text)

    # tokenize
    tokens = word_tokenize(text)

    # remove stopwords + short tokens + lemmatize
    clean_tokens = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token not in stop_words and len(token) > 2
    ]

    return clean_tokens



In [ ]:

tokens = preprocess_text(full_text)

print("Sample tokens:", tokens[:25])
print("Total tokens:", len(tokens))

Sample tokens: ['github', 'doc', 'get', 'started', 'troubleshoot', 'make', 'github', 'documentation', 'new', 'user', 'developer', 'administrator', 'github', 'product', 'github', 'secure', 'started', 'copilot', 'project', 'github', 'doc', 'help', 'wherever', 'github', 'journey']
Total tokens: 155


In [ ]:
from collections import defaultdict

inverted_index = defaultdict(list)
doc_id = 0   # change for multiple documents

for position, token in enumerate(tokens):
    inverted_index[token].append((doc_id, position))

# Example
print("Index for 'data':", inverted_index.get("data"))
print("Index size:", len(inverted_index))


Index for 'data': [(0, 105)]
Index size: 107


In [ ]:
inverted_index

defaultdict(list,
            {'github': [(0, 0),
              (0, 6),
              (0, 12),
              (0, 14),
              (0, 19),
              (0, 23),
              (0, 29),
              (0, 46),
              (0, 58),
              (0, 64),
              (0, 77),
              (0, 91),
              (0, 117),
              (0, 131),
              (0, 140)],
             'doc': [(0, 1), (0, 20), (0, 42), (0, 138), (0, 141)],
             'get': [(0, 2), (0, 25), (0, 119)],
             'started': [(0, 3), (0, 16), (0, 26), (0, 44)],
             'troubleshoot': [(0, 4)],
             'make': [(0, 5), (0, 137)],
             'documentation': [(0, 7)],
             'new': [(0, 8)],
             'user': [(0, 9)],
             'developer': [(0, 10), (0, 40)],
             'administrator': [(0, 11)],
             'product': [(0, 13)],
             'secure': [(0, 15), (0, 66), (0, 71), (0, 106)],
             'copilot': [(0, 17), (0, 30), (0, 118)],
             'project': [(0,

In [ ]:
import os
import json
import re
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# -----------------------
# Create data directory
# -----------------------
DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

INPUT_FILE = "search_index.jsonl"
OUTPUT_FILE = os.path.join(DATA_DIR, "inverted_index.json")

# -----------------------
# Load JSONL data
# -----------------------
json_data = []

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    for i, line in enumerate(f, start=1):
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
            json_data.append(obj)
        except json.JSONDecodeError as e:
            print(f"❌ Error parsing line {i}: {e}")

print("✅ Total records loaded:", len(json_data))

# -----------------------
# NLTK Setup
# -----------------------
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    tokens = word_tokenize(text)

    return [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token not in stop_words and len(token) > 2
    ]

# -----------------------
# Build Inverted Index
# -----------------------
inverted_index = defaultdict(list)

for doc_id, record in enumerate(json_data):
    title = record.get("title", "")
    description = record.get("description", "")
    keywords = record.get("keywords", "")
    visible_text = record.get("visible_text", "")

    full_text = f"{title} {description} {keywords} {visible_text}"

    tokens = preprocess_text(full_text)

    # Avoid duplicate doc_id per term
    for token in set(tokens):
        inverted_index[token].append(doc_id)

# Sort postings
for token in inverted_index:
    inverted_index[token] = sorted(inverted_index[token])

# -----------------------
# Save Index
# -----------------------
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(inverted_index, f, indent=2)

print(f"📁 Index saved in: {OUTPUT_FILE}")
print(f"📚 Total indexed terms:", len(inverted_index))


Exception ignored in: <function _on_executor_deleted at 0x0000022A6FB46290>
Traceback (most recent call last):
  File "d:\ir_mini_projecct\IR\venv\lib\site-packages\pymongo\periodic_executor.py", line 274, in _on_executor_deleted
    _EXECUTORS.remove(ref)
KeyboardInterrupt: 


✅ Total records loaded: 16742


In [ ]:
import json
import re
from pymongo import MongoClient
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from preprocess import preprocess_text
# -----------------------
# MongoDB Setup
# -----------------------
client = MongoClient("mongodb://localhost:27017/")
db = client["ir_database"]
docs_col = db["documents"]
index_col = db["inverted_index"]

# -----------------------
# NLTK Setup
# -----------------------
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# -----------------------
# Load JSONL and Insert/Update
# -----------------------
INPUT_FILE = "search_index.jsonl"

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            record = json.loads(line)
        except json.JSONDecodeError:
            continue

        url = record.get("url")
        if not url:
            continue

        # Combine all text fields
        title = record.get("title", "")
        description = record.get("description", "")
        keywords = " ".join(record.get("keywords", []))
        visible_text = record.get("visible_text", "")
        full_text = f"{title} {description} {keywords} {visible_text}"

        # Preprocess tokens
        tokens = preprocess_text(full_text)

        # Check if document exists
        existing_doc = docs_col.find_one({"url": url})

        if existing_doc:
            # Document exists, check content hash
            if existing_doc.get("content_hash") == record.get("content_hash"):
                # No changes, skip
                continue
            else:
                # Update document
                doc_id = existing_doc["_id"]

                # Remove old tokens from inverted index
                old_tokens = existing_doc.get("tokens", [])
                for token in set(old_tokens):
                    index_col.update_one(
                        {"token": token},
                        {"$pull": {"postings": doc_id}}
                    )

                # Update document fields
                docs_col.update_one(
                    {"_id": doc_id},
                    {"$set": {
                        "title": title,
                        "description": description,
                        "keywords": record.get("keywords", []),
                        "visible_text": visible_text,
                        "image_urls": record.get("image_urls", []),
                        "content_hash": record.get("content_hash"),
                        "crawled_at": record.get("crawled_at"),
                        "depth": record.get("depth"),
                        "tokens": tokens
                    }}
                )

        else:
            # Insert new document
            doc_id = docs_col.estimated_document_count() + 1
            docs_col.insert_one({
                "_id": doc_id,
                "url": url,
                "title": title,
                "description": description,
                "keywords": record.get("keywords", []),
                "visible_text": visible_text,
                "image_urls": record.get("image_urls", []),
                "content_hash": record.get("content_hash"),
                "crawled_at": record.get("crawled_at"),
                "depth": record.get("depth"),
                "tokens": tokens
            })

        # Update inverted index
        for token in set(tokens):
            index_col.update_one(
                {"token": token},
                {"$addToSet": {"postings": doc_id}},
                upsert=True
            )

print("✅ Documents updated/added and inverted index updated!")
print("📚 Total documents:", docs_col.count_documents({}))
print("📚 Total indexed terms:", index_col.count_documents({}))


In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor
from pymongo import MongoClient, UpdateOne
from preprocess import preprocess_text
from itertools import islice

# -----------------------
# Configuration
# -----------------------
INPUT_FILE = "batches/batch_1.json"
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "ir_database"

DOC_COLLECTION = "documents"
INDEX_COLLECTION = "inverted_index_posting_list"
COUNTER_COLLECTION = "counters"

BATCH_SIZE = 500
WORKERS = 8

# -----------------------
# MongoDB Setup
# -----------------------
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
docs_col = db[DOC_COLLECTION]
index_col = db[INDEX_COLLECTION]
counter_col = db[COUNTER_COLLECTION]

# -----------------------
# Ensure counter exists
# -----------------------
counter_col.update_one(
    {"_id": "doc_id"},
    {"$setOnInsert": {"seq": 0}},
    upsert=True
)

def get_next_doc_id():
    doc = counter_col.find_one_and_update(
        {"_id": "doc_id"},
        {"$inc": {"seq": 1}},
        return_document=True
    )
    return doc["seq"]

# -----------------------
# Chunk reader
# -----------------------
def chunked_iterator(file, size):
    while True:
        chunk = list(islice(file, size))
        if not chunk:
            break
        yield chunk

# -----------------------
# Worker: preprocess only (CPU safe)
# -----------------------
def process_lines(lines):
    batch = []

    for line in lines:
        try:
            record = json.loads(line)
        except Exception:
            continue

        url = record.get("url")
        if not url:
            continue

        title = record.get("title", "")
        description = record.get("description", "")
        keywords = " ".join(record.get("keywords", []))
        visible_text = record.get("visible_text", "")

        full_text = f"{title} {description} {keywords} {visible_text}"
        tokens = list(set(preprocess_text(full_text)))

        batch.append({
            "url": url,
            "tokens": tokens,
            "record": record
        })

    return batch

# -----------------------
# Main pipeline
# -----------------------
def main():
    print("🚀 Starting indexing...\n")

    with open(INPUT_FILE, "r", encoding="utf-8") as f, \
         ThreadPoolExecutor(max_workers=WORKERS) as pool:

        for batch_no, lines in enumerate(chunked_iterator(f, BATCH_SIZE), 1):

            future = pool.submit(process_lines, lines)
            processed = future.result()

            doc_ops = []
            index_ops = []

            for item in processed:
                url = item["url"]
                tokens = item["tokens"]
                record = item["record"]

                existing = docs_col.find_one(
                    {"url": url},
                    {"_id": 1, "content_hash": 1, "tokens": 1}
                )

                # ------------------
                # Skip unchanged docs
                # ------------------
                if existing and existing.get("content_hash") == record.get("content_hash"):
                    continue

                # ------------------
                # Assign doc_id
                # ------------------
                if existing:
                    doc_id = existing["_id"]

                    # Remove old tokens from inverted index
                    for old_token in existing.get("tokens", []):
                        index_ops.append(
                            UpdateOne(
                                {"token": old_token},
                                {"$pull": {"postings": doc_id}}
                            )
                        )
                else:
                    doc_id = get_next_doc_id()

                # ------------------
                # Upsert document
                # ------------------
                doc_ops.append(
                    UpdateOne(
                        {"_id": doc_id},
                        {"$set": {
                            "url": url,
                            "title": record.get("title", ""),
                            "description": record.get("description", ""),
                            "keywords": record.get("keywords", []),
                            "visible_text": record.get("visible_text", ""),
                            "image_urls": record.get("image_urls", []),
                            "content_hash": record.get("content_hash"),
                            "crawled_at": record.get("crawled_at"),
                            "depth": record.get("depth"),
                            "tokens": tokens,
                            "internal_links": record.get("internal_links",[]),
                        }},
                        upsert=True
                    )
                )

                # ------------------
                # Update inverted index
                # ------------------
                for token in tokens:
                    index_ops.append(
                        UpdateOne(
                            {"token": token},
                            {"$addToSet": {"postings": doc_id}},
                            upsert=True
                        )
                    )

            # ------------------
            # Bulk write
            # ------------------
            if doc_ops:
                docs_col.bulk_write(doc_ops, ordered=False)

            if index_ops:
                index_col.bulk_write(index_ops, ordered=False)

            print(f"✅ Batch {batch_no} indexed ({len(processed)} records)")

    print("\n🎉 Indexing completed!")
    print("📚 Total documents:", docs_col.count_documents({}))
    print("📚 Total indexed terms:", index_col.count_documents({}))

# -----------------------
# Run
# -----------------------
if __name__ == "__main__":
    main()


🚀 Starting indexing...



AttributeError: 'str' object has no attribute 'get'

In [17]:
import json
from concurrent.futures import ThreadPoolExecutor
from pymongo import MongoClient, UpdateOne
from preprocess import preprocess_text

# -----------------------
# Configuration
# -----------------------
start=17
end=36
for i in range(start,end):
    print(i)
    INPUT_FILE = f"batches/batch_{i}.json"
    MONGO_URI = "mongodb://localhost:27017/"
    DB_NAME = "ir_db"

    DOC_COLLECTION = "documents"
    INDEX_COLLECTION = "inverted_index"
    COUNTER_COLLECTION = "counters"

    BATCH_SIZE = 500
    WORKERS = 8

    # -----------------------
    # MongoDB Setup
    # -----------------------
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]

    docs_col = db[DOC_COLLECTION]
    index_col = db[INDEX_COLLECTION]
    counter_col = db[COUNTER_COLLECTION]

    # -----------------------
    # Ensure indexes (run once safely)
    # -----------------------
    docs_col.create_index("url", unique=True)
    index_col.create_index("token")

    # -----------------------
    # Ensure counter exists
    # -----------------------
    counter_col.update_one(
        {"_id": "doc_id"},
        {"$setOnInsert": {"seq": 0}},
        upsert=True
    )

    def get_next_doc_id():
        doc = counter_col.find_one_and_update(
            {"_id": "doc_id"},
            {"$inc": {"seq": 1}},
            return_document=True
        )
        return doc["seq"]

    # -----------------------
    # Chunk list helper
    # -----------------------
    def chunk_list(data, size):
        for i in range(0, len(data), size):
            yield data[i:i + size]

    # -----------------------
    # Worker: preprocess records
    # -----------------------
    def process_records(records):
        batch = []

        for record in records:
            url = record.get("url")
            if not url:
                continue

            title = record.get("title", "")
            description = record.get("description", "")
            keywords = " ".join(record.get("keywords", []))
            visible_text = record.get("visible_text", "")

            full_text = f"{title} {description} {keywords} {visible_text}"
            tokens = list(set(preprocess_text(full_text)))

            batch.append({
                "url": url,
                "tokens": tokens,
                "record": record
            })

        return batch

    # -----------------------
    # Main pipeline
    # -----------------------
    def main():
        print("🚀 Starting indexing...\n")

        # -----------------------
        # Load JSON batch file
        # -----------------------
        with open(INPUT_FILE, "r", encoding="utf-8") as f:
            all_records = json.load(f)

        print(f"📦 Loaded {len(all_records)} records from {INPUT_FILE}\n")

        # -----------------------
        # Process in batches
        # -----------------------
        with ThreadPoolExecutor(max_workers=WORKERS) as pool:

            for batch_no, records in enumerate(chunk_list(all_records, BATCH_SIZE), 1):

                # Optional: Deduplicate URLs inside this batch
                unique = {}
                for r in records:
                    if "url" in r:
                        unique[r["url"]] = r
                records = list(unique.values())

                future = pool.submit(process_records, records)
                processed = future.result()

                doc_ops = []
                index_ops = []

                for item in processed:
                    url = item["url"]
                    tokens = item["tokens"]
                    record = item["record"]

                    existing = docs_col.find_one(
                        {"url": url},
                        {"_id": 1, "content_hash": 1, "tokens": 1}
                    )

                    # ------------------
                    # Skip unchanged docs
                    # ------------------
                    if existing and existing.get("content_hash") == record.get("content_hash"):
                        continue

                    # ------------------
                    # Assign doc_id
                    # ------------------
                    if existing:
                        doc_id = existing["_id"]

                        # Remove old tokens from inverted index
                        for old_token in existing.get("tokens", []):
                            index_ops.append(
                                UpdateOne(
                                    {"token": old_token},
                                    {"$pull": {"postings": doc_id}}
                                )
                            )
                    else:
                        doc_id = get_next_doc_id()

                    # ------------------
                    # ✅ FIXED: Upsert document using URL
                    # ------------------
                    doc_ops.append(
                        UpdateOne(
                            {"url": url},     # ✅ match by URL (not _id)
                            {"$set": {
                                "_id": doc_id,
                                "url": url,
                                "title": record.get("title", ""),
                                "description": record.get("description", ""),
                                "keywords": record.get("keywords", []),
                                "visible_text": record.get("visible_text", ""),
                                "image_urls": record.get("image_urls", []),
                                "content_hash": record.get("content_hash"),
                                "crawled_at": record.get("crawled_at"),
                                "depth": record.get("depth"),
                                "tokens": tokens,
                                "internal_links": record.get("internal_links", []),
                            }},
                            upsert=True
                        )
                    )

                    # ------------------
                    # Update inverted index
                    # ------------------
                    for token in tokens:
                        index_ops.append(
                            UpdateOne(
                                {"token": token},
                                {"$addToSet": {"postings": doc_id}},
                                upsert=True
                            )
                        )

                # ------------------
                # Bulk write to MongoDB
                # ------------------
                if doc_ops:
                    docs_col.bulk_write(doc_ops, ordered=False)

                if index_ops:
                    index_col.bulk_write(index_ops, ordered=False)

                print(f"✅ Batch {batch_no} indexed ({len(processed)} records)")

        print("\n🎉 Indexing completed!")
        print("📚 Total documents:", docs_col.count_documents({}))
        print("📚 Total indexed terms:", index_col.count_documents({}))

    # -----------------------
    # Run
    # -----------------------
    if __name__ == "__main__":
        main()


17
🚀 Starting indexing...

📦 Loaded 1000 records from batches/batch_17.json

✅ Batch 1 indexed (500 records)
✅ Batch 2 indexed (500 records)

🎉 Indexing completed!
📚 Total documents: 16742
📚 Total indexed terms: 124275
18
🚀 Starting indexing...

📦 Loaded 1000 records from batches/batch_18.json

✅ Batch 1 indexed (500 records)
✅ Batch 2 indexed (500 records)

🎉 Indexing completed!
📚 Total documents: 17052
📚 Total indexed terms: 125667
19
🚀 Starting indexing...

📦 Loaded 1000 records from batches/batch_19.json

✅ Batch 1 indexed (500 records)
✅ Batch 2 indexed (499 records)

🎉 Indexing completed!
📚 Total documents: 17572
📚 Total indexed terms: 129091
20
🚀 Starting indexing...

📦 Loaded 1000 records from batches/batch_20.json

✅ Batch 1 indexed (500 records)
✅ Batch 2 indexed (500 records)

🎉 Indexing completed!
📚 Total documents: 18499
📚 Total indexed terms: 134129
21
🚀 Starting indexing...

📦 Loaded 1000 records from batches/batch_21.json

✅ Batch 1 indexed (500 records)
✅ Batch 2 index

In [1]:
import json
import os

INPUT_FILE = "search_index.jsonl"
OUTPUT_DIR = "batches"
BATCH_SIZE = 1000

os.makedirs(OUTPUT_DIR, exist_ok=True)

batch = []
batch_id = 1

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    for line_num, line in enumerate(f, start=1):
        line = line.strip()
        if not line:
            continue

        try:
            record = json.loads(line)
            batch.append(record)
        except json.JSONDecodeError:
            print(f"❌ Skipping invalid JSON at line {line_num}")
            continue

        if len(batch) == BATCH_SIZE:
            output_path = f"{OUTPUT_DIR}/batch_{batch_id}.json"
            with open(output_path, "w", encoding="utf-8") as out:
                json.dump(batch, out, indent=2)

            print(f"✅ Saved {output_path} ({len(batch)} records)")
            batch.clear()
            batch_id += 1

# Save remaining records
if batch:
    output_path = f"{OUTPUT_DIR}/batch_{batch_id}.json"
    with open(output_path, "w", encoding="utf-8") as out:
        json.dump(batch, out, indent=2)

    print(f"✅ Saved {output_path} ({len(batch)} records)")


✅ Saved batches/batch_1.json (1000 records)
✅ Saved batches/batch_2.json (1000 records)
✅ Saved batches/batch_3.json (1000 records)
✅ Saved batches/batch_4.json (1000 records)
✅ Saved batches/batch_5.json (1000 records)
✅ Saved batches/batch_6.json (1000 records)
✅ Saved batches/batch_7.json (1000 records)
✅ Saved batches/batch_8.json (1000 records)
✅ Saved batches/batch_9.json (1000 records)
✅ Saved batches/batch_10.json (1000 records)
✅ Saved batches/batch_11.json (1000 records)
✅ Saved batches/batch_12.json (1000 records)
✅ Saved batches/batch_13.json (1000 records)
✅ Saved batches/batch_14.json (1000 records)
✅ Saved batches/batch_15.json (1000 records)
✅ Saved batches/batch_16.json (1000 records)
✅ Saved batches/batch_17.json (1000 records)
✅ Saved batches/batch_18.json (1000 records)
✅ Saved batches/batch_19.json (1000 records)
✅ Saved batches/batch_20.json (1000 records)
✅ Saved batches/batch_21.json (1000 records)
✅ Saved batches/batch_22.json (1000 records)
✅ Saved batches/bat

In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor
from pymongo import MongoClient, UpdateOne
from preprocess import preprocess_text
from collections import Counter

# -----------------------
# Configuration
# -----------------------
start = 17
end = 36

MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "ir_db"

DOC_COLLECTION = "documents_1"
INDEX_COLLECTION = "inverted_index_tf"
COUNTER_COLLECTION = "counters_1"

BATCH_SIZE = 500
WORKERS = 8

# -----------------------
# MongoDB Setup
# -----------------------
client = MongoClient(MONGO_URI)
db = client[DB_NAME]

docs_col = db[DOC_COLLECTION]
index_col = db[INDEX_COLLECTION]
counter_col = db[COUNTER_COLLECTION]

# -----------------------
# Ensure indexes (run once safely)
# -----------------------
docs_col.create_index("url", unique=True)
index_col.create_index("token")

# -----------------------
# Ensure counter exists
# -----------------------
counter_col.update_one(
    {"_id": "doc_id"},
    {"$setOnInsert": {"seq": 0}},
    upsert=True
)

def get_next_doc_id():
    doc = counter_col.find_one_and_update(
        {"_id": "doc_id"},
        {"$inc": {"seq": 1}},
        return_document=True
    )
    return doc["seq"]

# -----------------------
# Chunk list helper
# -----------------------
def chunk_list(data, size):
    for i in range(0, len(data), size):
        yield data[i:i + size]

# -----------------------
# Worker: preprocess records
# -----------------------
def process_records(records):
    batch = []

    for record in records:
        url = record.get("url")
        if not url:
            continue

        title = record.get("title", "")
        description = record.get("description", "")
        keywords = " ".join(record.get("keywords", []))
        visible_text = record.get("visible_text", "")

        full_text = f"{title} {description} {keywords} {visible_text}"

        token_list = preprocess_text(full_text)     # keep duplicates
        tf_map = Counter(token_list)                # count TF

        batch.append({
            "url": url,
            "tf_map": tf_map,
            "record": record
        })

    return batch

# -----------------------
# Main pipeline
# -----------------------
def main():

    print("🚀 Starting indexing...\n")

    for i in range(start, end):
        INPUT_FILE = f"batches/batch_{i}.json"
        print(f"\n📂 Processing: {INPUT_FILE}")

        # -----------------------
        # Load JSON batch file
        # -----------------------
        with open(INPUT_FILE, "r", encoding="utf-8") as f:
            all_records = json.load(f)

        print(f"📦 Loaded {len(all_records)} records")

        # -----------------------
        # Process in batches
        # -----------------------
        with ThreadPoolExecutor(max_workers=WORKERS) as pool:

            for batch_no, records in enumerate(chunk_list(all_records, BATCH_SIZE), 1):

                # Deduplicate URLs inside batch
                unique = {}
                for r in records:
                    if "url" in r:
                        unique[r["url"]] = r
                records = list(unique.values())

                future = pool.submit(process_records, records)
                processed = future.result()

                doc_ops = []
                index_ops = []

                for item in processed:
                    url = item["url"]
                    tf_map = item["tf_map"]
                    tokens = list(tf_map.keys())
                    record = item["record"]

                    existing = docs_col.find_one(
                        {"url": url},
                        {"_id": 1, "content_hash": 1, "tokens": 1}
                    )

                    # ------------------
                    # Skip unchanged docs
                    # ------------------
                    if existing and existing.get("content_hash") == record.get("content_hash"):
                        continue

                    # ------------------
                    # Assign doc_id
                    # ------------------
                    if existing:
                        doc_id = existing["_id"]

                        # Remove old postings from inverted index
                        for old_token in existing.get("tokens", []):
                            index_ops.append(
                                UpdateOne(
                                    {"token": old_token},
                                    {"$pull": {"postings": {"doc_id": doc_id}}}
                                )
                            )
                    else:
                        doc_id = get_next_doc_id()

                    # ------------------
                    # Upsert document
                    # ------------------
                    doc_ops.append(
                        UpdateOne(
                            {"url": url},
                            {"$set": {
                                "_id": doc_id,
                                "url": url,
                                "title": record.get("title", ""),
                                "description": record.get("description", ""),
                                "keywords": record.get("keywords", []),
                                "visible_text": record.get("visible_text", ""),
                                "image_urls": record.get("image_urls", []),
                                "content_hash": record.get("content_hash"),
                                "crawled_at": record.get("crawled_at"),
                                "depth": record.get("depth"),
                                "tokens": tokens,          # unique tokens only
                                "internal_links": record.get("internal_links", []),
                            }},
                            upsert=True
                        )
                    )

                    # ------------------
                    # Update inverted index with TF
                    # ------------------
                    for token, tf in tf_map.items():
                        index_ops.append(
                            UpdateOne(
                                {"token": token},
                                {"$addToSet": {
                                    "postings": {
                                        "doc_id": doc_id,
                                        "tf": tf
                                    }
                                }},
                                upsert=True
                            )
                        )

                # ------------------
                # Bulk write to MongoDB
                # ------------------
                if doc_ops:
                    docs_col.bulk_write(doc_ops, ordered=False)

                if index_ops:
                    index_col.bulk_write(index_ops, ordered=False)

                print(f"✅ Batch {batch_no} indexed ({len(processed)} docs)")

    print("\n🎉 Indexing completed!")
    print("📚 Total documents:", docs_col.count_documents({}))
    print("📚 Total indexed terms:", index_col.count_documents({}))


# -----------------------
# Run
# -----------------------
if __name__ == "__main__":
    main()


🚀 Starting indexing...


📂 Processing: batches/batch_17.json
📦 Loaded 1000 records
✅ Batch 1 indexed (500 docs)
✅ Batch 2 indexed (500 docs)

📂 Processing: batches/batch_18.json
📦 Loaded 1000 records
✅ Batch 1 indexed (500 docs)
✅ Batch 2 indexed (500 docs)

📂 Processing: batches/batch_19.json
📦 Loaded 1000 records
✅ Batch 1 indexed (500 docs)
✅ Batch 2 indexed (499 docs)

📂 Processing: batches/batch_20.json
📦 Loaded 1000 records
✅ Batch 1 indexed (500 docs)
✅ Batch 2 indexed (500 docs)

📂 Processing: batches/batch_21.json
📦 Loaded 1000 records
✅ Batch 1 indexed (500 docs)
✅ Batch 2 indexed (500 docs)

📂 Processing: batches/batch_22.json
📦 Loaded 1000 records
✅ Batch 1 indexed (500 docs)
✅ Batch 2 indexed (499 docs)

📂 Processing: batches/batch_23.json
📦 Loaded 1000 records
✅ Batch 1 indexed (500 docs)
